Fake News Using Bidirectional LSTM

In [32]:
import pandas as pd

In [33]:
df = pd.read_csv('train.csv')

In [34]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [35]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [36]:
df = df.dropna()

In [37]:
X = df.drop('label', axis = 1)
y = df['label']

In [38]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [39]:
import tensorflow as tf

In [40]:
tf.__version__

'2.12.0'

In [41]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Dense, Bidirectional

In [42]:
voc_size = 5000

In [43]:
messages = X.copy()
messages.reset_index(inplace = True)

In [44]:
import nltk
import re
from nltk.corpus import stopwords

In [45]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [47]:
onehot_rep = [one_hot(words, voc_size) for words in corpus]

In [48]:
onehot_rep

[[3463, 3880, 3486, 2212, 2748, 3674, 954, 1686, 2567, 4560],
 [2201, 4936, 689, 3503, 1990, 4545, 3288],
 [399, 880, 2711, 1702],
 [2776, 228, 4784, 299, 4164, 2075],
 [589, 1990, 3306, 3606, 15, 1017, 1990, 869, 4039, 2418],
 [4507,
  1981,
  3108,
  3707,
  3750,
  4439,
  4116,
  852,
  1115,
  2665,
  3981,
  3625,
  4664,
  4660,
  3288],
 [1414, 2659, 4848, 683, 527, 594, 4939, 3500, 3974, 4759, 2762],
 [558, 1527, 127, 3303, 395, 2255, 4439, 541, 3974, 4759, 2762],
 [3264, 4373, 3030, 4782, 1246, 3553, 4888, 3238, 4439, 660],
 [3830, 814, 2835, 1989, 484, 4118, 3018, 1466],
 [1231, 2057, 2450, 2538, 768, 384, 4159, 2793, 3736, 1479, 463],
 [299, 3450, 2748, 3553, 4439, 395],
 [647, 2557, 4452, 208, 591, 2584, 1830, 4188, 2046],
 [607, 2533, 1144, 3130, 4608, 1947, 4891, 3974, 4759, 2762],
 [4992, 4381, 2555, 4607, 4444, 3974, 4759, 2762],
 [357, 2399, 817, 2567, 4961, 1780, 836, 942, 1872, 909],
 [3226, 4380, 4936],
 [1115, 753, 1167, 952, 4439, 4177, 4073, 3288],
 [3009, 1489,

Embedding Rep

In [49]:
sent_length = 20
embedded_docs = pad_sequences(onehot_rep, padding = 'pre', maxlen = sent_length)

In [50]:
embedded_docs

array([[   0,    0,    0, ..., 1686, 2567, 4560],
       [   0,    0,    0, ..., 1990, 4545, 3288],
       [   0,    0,    0, ...,  880, 2711, 1702],
       ...,
       [   0,    0,    0, ..., 3974, 4759, 2762],
       [   0,    0,    0, ...,   54,  745,   32],
       [   0,    0,    0, ..., 4138, 1498, 2701]], dtype=int32)

In [51]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer ='adam', metrics = ['accuracy'])

In [52]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [53]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [55]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 15, batch_size = 32)

Epoch 1/15
383/383 [==============================] - 28s 48ms/step - loss: 0.2643 - accuracy: 0.8779 - val_loss: 0.1921 - val_accuracy: 0.9210
Epoch 2/15
383/383 [==============================] - 7s 18ms/step - loss: 0.1320 - accuracy: 0.9505 - val_loss: 0.2083 - val_accuracy: 0.9157
Epoch 3/15
383/383 [==============================] - 4s 10ms/step - loss: 0.0894 - accuracy: 0.9691 - val_loss: 0.2285 - val_accuracy: 0.9185
Epoch 4/15
383/383 [==============================] - 4s 10ms/step - loss: 0.0587 - accuracy: 0.9806 - val_loss: 0.2486 - val_accuracy: 0.9175
Epoch 5/15
383/383 [==============================] - 5s 13ms/step - loss: 0.0397 - accuracy: 0.9871 - val_loss: 0.3810 - val_accuracy: 0.9079
Epoch 6/15
383/383 [==============================] - 4s 10ms/step - loss: 0.0276 - accuracy: 0.9917 - val_loss: 0.3788 - val_accuracy: 0.9100
Epoch 7/15
383/383 [==============================] - 4s 10ms/step - loss: 0.0210 - accuracy: 0.9935 - val_loss: 0.4268 - val_accuracy: 0.913

In [56]:
y_pred = model.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [59]:
y_pred = np.where(y_pred >= 0.5, 1, 0)

In [62]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(y_test, y_pred)

array([[3128,  291],
       [ 250, 2366]])

In [63]:
accuracy_score(y_test, y_pred)

0.9103562551781276